In [1]:
import time
import numpy as np
import ROOT

canvas = ROOT.TCanvas()

Welcome to JupyROOT 6.24/00


In [2]:
rootfile = ROOT.TFile.Open("data/HiggsZZ4mu.root")
roottree = rootfile.Get("Events")

In [ ]:
starttime = time.time()

roothist = ROOT.TH1D("roothist", "mass", 120, 0, 120)

for index, event in enumerate(roottree):
    # Applying cuts means if-statements.
    if event.nMuon >= 2 and event.Muon_charge[0] + event.Muon_charge[1] == 0:
        mu1_pt = event.Muon_pt[0]
        mu2_pt = event.Muon_pt[1]
        mu1_eta = event.Muon_eta[0]
        mu2_eta = event.Muon_eta[1]
        mu1_phi = event.Muon_phi[0]
        mu2_phi = event.Muon_phi[1]
        roothist.Fill(
            np.sqrt(2*mu1_pt*mu2_pt*(np.cosh(mu1_eta - mu2_eta) - np.cos(mu1_phi - mu2_phi)))
        )

pyroot_time = time.time() - starttime
print(f"total time: {pyroot_time} sec")

In [ ]:
roothist.Draw()
canvas.Draw()

In [ ]:
ROOT.gInterpreter.Declare('''
void compute(TH1D& roothist, TTree& roottree) {
    UInt_t nMuon;
    float Muon_pt[50];
    float Muon_eta[50];
    float Muon_phi[50];
    int32_t Muon_charge[50];

    roottree.SetBranchStatus("*", 0);
    roottree.SetBranchStatus("nMuon", 1);
    roottree.SetBranchStatus("Muon_pt", 1);
    roottree.SetBranchStatus("Muon_eta", 1);
    roottree.SetBranchStatus("Muon_phi", 1);
    roottree.SetBranchStatus("Muon_charge", 1);

    roottree.SetBranchAddress("nMuon", &nMuon);
    roottree.SetBranchAddress("Muon_pt", Muon_pt);
    roottree.SetBranchAddress("Muon_eta", Muon_eta);
    roottree.SetBranchAddress("Muon_phi", Muon_phi);
    roottree.SetBranchAddress("Muon_charge", Muon_charge);

    for (int index = 0; index < 100000; index++) {
        roottree.GetEntry(index);
        if (nMuon >= 2 && Muon_charge[0] + Muon_charge[1] == 0) {
            float mu1_pt = Muon_pt[0];
            float mu2_pt = Muon_pt[1];
            float mu1_eta = Muon_eta[0];
            float mu2_eta = Muon_eta[1];
            float mu1_phi = Muon_phi[0];
            float mu2_phi = Muon_phi[1];
            roothist.Fill(
                sqrt(2*mu1_pt*mu2_pt*(cosh(mu1_eta - mu2_eta) - cos(mu1_phi - mu2_phi)))
            );
        }
    }
}
''')

In [ ]:
starttime = time.time()

roothist2 = ROOT.TH1D("roothist2", "mass", 120, 0, 120)

ROOT.compute(roothist2, roottree)

cpproot_time = time.time() - starttime
print(f"total time: {cpproot_time} sec")

In [ ]:
pyroot_time / cpproot_time

In [ ]:
roothist2.Draw()
canvas.Draw()

<img src="img/rdataframe-flow.svg" style="width: 800px">

In [ ]:
df = ROOT.RDataFrame("Events", "data/HiggsZZ4mu.root")

# Each node is connected to the previous, in a chain (which can split and recombine).
df_2mu = df.Filter("nMuon >= 2")
df_os = df_2mu.Filter("Muon_charge[0] + Muon_charge[1] == 0")

# This node is a big C++ block.
df_mass = df_os.Define("Dimuon_mass", '''
float mu1_pt = Muon_pt[0];
float mu2_pt = Muon_pt[1];
float mu1_eta = Muon_eta[0];
float mu2_eta = Muon_eta[1];
float mu1_phi = Muon_phi[0];
float mu2_phi = Muon_phi[1];
return sqrt(2*mu1_pt*mu2_pt*(cosh(mu1_eta - mu2_eta) - cos(mu1_phi - mu2_phi)));
''')

In [ ]:
starttime = time.time()

# This one is an endpoint (action).
roothist3 = df_mass.Histo1D(("h3", "mass", 120, 0, 120), "Dimuon_mass")

rdfroot_time = time.time() - starttime
print(f"total time: {rdfroot_time} sec")

In [ ]:
pyroot_time / rdfroot_time

In [ ]:
roothist3.Draw()
canvas.Draw()

In [3]:
import awkward as ak
import uproot
import hist

In [ ]:
events = uproot.open("data/HiggsZZ4mu.root:Events")
events.show()

In [ ]:
muons = events.arrays(
    ["pt", "eta", "phi", "charge"],
    aliases={"pt": "Muon_pt", "eta": "Muon_eta", "phi": "Muon_phi", "charge": "Muon_charge"}
)
muons

In [ ]:
cut = (ak.num(muons.charge) >= 2) & (ak.sum(muons.charge[:, :2], axis=1) == 0)
cut

In [ ]:
mu1 = muons[cut, 0]
mu2 = muons[cut, 1]
mu1, mu2

In [ ]:
h = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()
h

In [ ]:
h.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

In [4]:
import matplotlib.pyplot as plt

In [ ]:
h.plot();

In [ ]:
starttime = time.time()

# read data
muons = events.arrays(
    ["pt", "eta", "phi", "charge"],
    aliases={"pt": "Muon_pt", "eta": "Muon_eta", "phi": "Muon_phi", "charge": "Muon_charge"},
    array_cache=None,   # no cheating!
)

# compute
cut = (ak.num(muons.charge) >= 2) & (ak.sum(muons.charge[:, :2], axis=1) == 0)
mu1 = muons[cut, 0]
mu2 = muons[cut, 1]
h = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()
h.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

uproot_time = time.time() - starttime
print(f"total time: {uproot_time} sec")

In [ ]:
pyroot_time / uproot_time

<img src="img/terminology.svg" style="width: 800px">

In [ ]:
histograms = uproot.open("data/HiggsZZ4mu_histograms.root")

In [ ]:
histograms.keys()

In [ ]:
histograms["Z/all/massZto2muon"].to_hist().plot()

In [ ]:
# find the 2D histogram and plot it

In [ ]:
icecube = uproot.open("data/icecube-supernovae.root")
icecube.keys()

In [ ]:
icecube.classname_of("config/detector")

In [ ]:
icecube.file.show_streamers("I3Eval_t")

In [ ]:
icecube["config/detector"]

In [ ]:
icecube["config/detector"].all_members

In [ ]:
icecube["config/detector"].member("ChannelIDMap")

In [ ]:
zmumu_file = uproot.open("data/Zmumu.root")
zmumu_file.keys()

In [ ]:
zmumu = zmumu_file["events"]
zmumu.show()

In [ ]:
zmumu.keys()

In [ ]:
zmumu.keys(filter_typename="/int.*/")

In [ ]:
zmumu.typenames()

In [ ]:
{name: branch.interpretation for name, branch in zmumu.items()}

In [ ]:
zmumu["M"].array()

Some important parameters:

   * `entry_start`, `entry_stop` to limit how much you read (if it's big)
   * `library="np"` for NumPy arrays, `library="ak"` for Awkward Arrays, and `library="pd"` for Pandas (Series or DataFrame)

In [ ]:
zmumu["M"].array(entry_stop=5)

In [ ]:
zmumu["M"].array(library="np")

In [ ]:
zmumu["M"].array(library="ak")   # default

In [ ]:
zmumu["M"].array(library="pd")

In [ ]:
zmumu.arrays()

In [ ]:
zmumu.arrays(library="np")

In [ ]:
zmumu.arrays(library="pd")

In [ ]:
zmumu.arrays(["px1", "py1", "px2", "py2"], library="pd")

Or it can be computed expressions.

In [ ]:
zmumu.arrays(["sqrt(px1**2 + py1**2)", "sqrt(px2**2 + py2**2)"], library="pd")

This is to support any [aliases](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#aliases) that might be in the [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html), but you can make up your own `aliases` on the spot.

In [ ]:
zmumu.arrays(["pt1", "pt2"], {"pt1": "sqrt(px1**2 + py1**2)", "pt2": "sqrt(px2**2 + py2**2)"}, library="pd")

The `expressions` parameter is not a good way to select branches by name.

   * nested branches, paths with "`/`", _would be interpreted as division!_
   * wildcards, paths with "`*`", _would be interpreted as multiplication!_

To select branches by name, use `filter_name`, `filter_typename`, `filter_branch` (all in the [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#arrays) documentation).

In [ ]:
zmumu.arrays(filter_name="p[xyz]*", library="pd")

(These filters have the same meaning as in [keys](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#keys) and [typenames](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#typenames), but those methods do not read potentially large datasets.)

In [ ]:
zmumu.keys(filter_name="p[xyz]*")

In [ ]:
zmumu.typenames(filter_name="p[xyz]*")

<br><br><br>

### Get arrays in manageable chunks

The [iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#iterate) method is like [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#arrays), but it can be used in a loop over chunks of the array.

How large are the chunks? You should set that with `step_size`.

In [ ]:
for arrays in zmumu.iterate(step_size=300):
    print(repr(arrays))

In [ ]:
for arrays in zmumu.iterate(step_size="50 kB"):   # 50 kB is very small! for illustrative purposes only!
    print(repr(arrays))

<br><br><br>

### Collections of files (like TChain)

If you want to read a bunch of files in one call, it has to be a function, rather than a method of [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html).

   * The equivalent of [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html) [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#arrays) is [uproot.concatenate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.concatenate.html). _(Reads everything at once: use this as a convenience on datasets you know are small!)_
   * The equivalent of [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html) [iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#iterate) is [uproot.iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.iterate.html). _(This is the most useful one.)_
   * There's also an [uproot.lazy](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.lazy.html) _(More on this below.)_

In [5]:
import IPython
import matplotlib.pyplot as plt
import matplotlib.pylab

In [ ]:
h = hist.Hist.new.Reg(100, 0, 500, name="mass").Double()

for muons in uproot.iterate(
    # filename(s)
    ["root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root:Events"],

    # expressions
    ["pt", "eta", "phi", "charge"],
    aliases={"pt": "Muon_pt", "eta": "Muon_eta", "phi": "Muon_phi", "charge": "Muon_charge"},    

    # the all-important step_size!
    step_size="1 MB",
):
    # do everything you're going to do to this array
    cut = (ak.num(muons.charge) >= 2) & (ak.sum(muons.charge[:, :2], axis=1) == 0)
    mu1 = muons[cut, 0]
    mu2 = muons[cut, 1]

    # such as filling a histogram
    h.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

    h.plot()
    plt.yscale("log")
    IPython.display.display(matplotlib.pylab.gcf())
    IPython.display.clear_output(wait=True)

    if h.counts().sum() > 300000:
        break

In [ ]:
lazy = uproot.lazy(
    # filename(s)
    ["root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root:Events"],
    # step_size is still important
    step_size="1 MB",
)
lazy

In [ ]:
lazy.Muon_pt

In [ ]:
lazy.Muon_eta

In [6]:
events = uproot.open("data/HiggsZZ4mu.root:Events")

In [ ]:
nonjagged = events["MET_pt"].array(entry_stop=20, library="np")
nonjagged

In [ ]:
nonjagged[:5]

In [ ]:
jagged_awkward = events["Muon_pt"].array(entry_stop=20)
jagged_awkward

In [ ]:
jagged_awkward.tolist()

In [ ]:
jagged_numpy = events["Muon_pt"].array(entry_stop=20, library="np")
jagged_numpy

In [ ]:
jagged_awkward[:, :1]

In [ ]:
jagged_numpy[:, :1]

In [ ]:
np.array([x[:1] for x in jagged_numpy])

In [ ]:
events.arrays(filter_name=["Muon_*"], library="pd")

In [7]:
array = ak.from_parquet("data/HiggsZZ4mu.parquet")
array

<Array [{run: 1, luminosityBlock: 13, ... ] type='299683 * {"run": int32, "lumin...'>

In [8]:
array.fields

['run', 'luminosityBlock', 'event', 'MET', 'muons', 'gen']

In [9]:
array[0].tolist()

{'run': 1,
 'luminosityBlock': 13,
 'event': 1201,
 'MET': {'pt': 19.49629020690918, 'phi': 3.096665859222412},
 'muons': [],
 'gen': [{'pt': 60.43461608886719,
   'eta': -0.7820958495140076,
   'phi': -2.21305251121521,
   'pdgId': 11},
  {'pt': 27.03217887878418,
   'eta': -2.3512237071990967,
   'phi': -0.6086280941963196,
   'pdgId': -11},
  {'pt': 60.43461608886719,
   'eta': -0.7820958495140076,
   'phi': -2.21305251121521,
   'pdgId': 11},
  {'pt': 8.345659255981445,
   'eta': -2.0826632976531982,
   'phi': 0.020737886428833008,
   'pdgId': -15},
  {'pt': 27.03217887878418,
   'eta': -2.3512237071990967,
   'phi': -0.6086280941963196,
   'pdgId': -11},
  {'pt': 8.345659255981445,
   'eta': -2.0826632976531982,
   'phi': 0.020737886428833008,
   'pdgId': -15}]}

In [10]:
array.muons.pt

<Array [[], [18.6, 23.6, ... 36.4, 19.6, 22.6]] type='299683 * var * float32'>

In [11]:
nanoaod_style = events.arrays(filter_name="Muon_*")
nanoaod_style.type

299683 * {"Muon_pt": var * float32, "Muon_eta": var * float32, "Muon_phi": var * float32, "Muon_mass": var * float32, "Muon_charge": var * int32}

In [12]:
array.muons.type

299683 * var * {"pt": float32, "eta": float32, "phi": float32, "mass": float32, "charge": int32}

In [13]:
nanoevents_style = ak.zip({
    "pt": nanoaod_style.Muon_pt,
    "eta": nanoaod_style.Muon_eta,
    "phi": nanoaod_style.Muon_phi,
    "mass": nanoaod_style.Muon_mass,
    "charge": nanoaod_style.Muon_charge,
})
nanoevents_style.type

299683 * var * {"pt": float32, "eta": float32, "phi": float32, "mass": float32, "charge": int32}

In [14]:
array.luminosityBlock

<Array [13, 13, 13, 13, ... 2801, 2801, 2801] type='299683 * int64'>

In [15]:
lumilengths = ak.run_lengths(array.luminosityBlock)
lumilengths

<Array [100, 100, 100, 100, ... 100, 100, 100] type='2997 * int64'>

In [16]:
array_by_lumi = ak.unflatten(array, lumilengths, axis=0)
array_by_lumi

<Array [[{run: 1, luminosityBlock: 13, ... ] type='2997 * var * {"run": int32, "...'>

In [17]:
array_by_lumi.luminosityBlock[0]

<Array [13, 13, 13, 13, 13, ... 13, 13, 13, 13] type='100 * int64'>

In [18]:
array.type

299683 * {"run": int32, "luminosityBlock": int64, "event": uint64, "MET": {"pt": float32, "phi": float32}, "muons": var * {"pt": float32, "eta": float32, "phi": float32, "mass": float32, "charge": int32}, "gen": var * {"pt": float32, "eta": float32, "phi": float32, "pdgId": int32}}

In [19]:
array_by_lumi.type

2997 * var * {"run": int32, "luminosityBlock": int64, "event": uint64, "MET": {"pt": float32, "phi": float32}, "muons": var * {"pt": float32, "eta": float32, "phi": float32, "mass": float32, "charge": int32}, "gen": var * {"pt": float32, "eta": float32, "phi": float32, "pdgId": int32}}

In [22]:
ak.sum(array_by_lumi.muons.pt, axis=-1)

<Array [[0, 42.2, 0, ... 26.5, 91.5, 109]] type='2997 * var * float32'>

<img src="img/example-reduction.svg" style="width: 800px">

In [23]:
array.muons

<Array [[], [{pt: 18.6, ... charge: -1}]] type='299683 * var * {"pt": float32, "...'>

In [24]:
cut = ak.num(array.muons) >= 2
cut

<Array [False, True, False, ... True, True] type='299683 * bool'>

In [25]:
array.muons[cut]

<Array [[{pt: 18.6, ... charge: -1}]] type='144474 * var * {"pt": float32, "eta"...'>

In [26]:
array.muons.mask[cut]

<Array [None, [{pt: 18.6, ... charge: -1}]] type='299683 * option[var * {"pt": f...'>

In [27]:
selected_muons = array.muons[cut]

selected_muons.charge[:, 0] + selected_muons.charge[:, 1] == 0

<Array [True, True, True, ... True, True, True] type='144474 * bool'>

<table style="margin-left: 0px">
    <tr style="background: white"><td style="font-size: 1.75em; font-weight: bold; text-align: center"><a href="https://awkward-array.readthedocs.io/en/latest/_auto/ak.cartesian.html">ak.cartesian</a></td><td style="font-size: 1.75em; font-weight: bold; text-align: center"><a href="https://awkward-array.readthedocs.io/en/latest/_auto/ak.combinations.html">ak.combinations</a></td></tr>
    <tr style="background: white"><td><img src="img/cartoon-cartesian.svg"></td><td><img src="img/cartoon-combinations.svg"></td></tr>
</table>

In [28]:
ak.combinations(array.muons, 2).type

299683 * var * ({"pt": float32, "eta": float32, "phi": float32, "mass": float32, "charge": int32}, {"pt": float32, "eta": float32, "phi": float32, "mass": float32, "charge": int32})

In [29]:
mu1, mu2 = ak.unzip(ak.combinations(array.muons, 2))
mu1, mu2

(<Array [[], [{pt: 18.6, ... charge: 1}]] type='299683 * var * {"pt": float32, "e...'>,
 <Array [[], [{pt: 23.6, ... charge: -1}]] type='299683 * var * {"pt": float32, "...'>)

In [30]:
h = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()
h.fill(ak.flatten(
    np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi)))
))

Hist(Regular(120, 0, 120, name='mass', label='mass'), storage=Double()) # Sum: 355686.0 (356468.0 with flow)

In [31]:
import vector
vector.register_awkward()

In [32]:
muons = ak.with_name(array.muons, "Momentum4D")
muons

<MomentumArray4D [[], [{pt: 18.6, ... charge: -1}]] type='299683 * var * Momentu...'>

In [33]:
mu1, mu2 = ak.unzip(ak.combinations(muons, 2))
mu1, mu2

(<MomentumArray4D [[], [{pt: 18.6, ... charge: 1}]] type='299683 * var * Momentum...'>,
 <MomentumArray4D [[], [{pt: 23.6, ... charge: -1}]] type='299683 * var * Momentu...'>)

In [34]:
(mu1 + mu2).mass

<Array [[], [36.9], ... 68, 48.8, 36.5, 46.9]] type='299683 * var * float32'>

In [35]:
h = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()
h.fill(ak.flatten((mu1 + mu2).mass))

Hist(Regular(120, 0, 120, name='mass', label='mass'), storage=Double()) # Sum: 355651.0 (356468.0 with flow)

In [36]:
gen = ak.with_name(array.gen, "Momentum3D")
gen

<MomentumArray3D [[{pt: 60.4, ... pdgId: 13}]] type='299683 * var * Momentum3D["...'>

In [37]:
gen.fields

['pt', 'eta', 'phi', 'pdgId']

In [38]:
reco_gen = ak.cartesian({"muon": muons, "gen": gen}, nested=True)
reco_gen

<Array [[], [[{muon: {, ... pdgId: 13}}]]] type='299683 * var * var * {"muon": M...'>

In [39]:
mu, g = ak.unzip(reco_gen)
mu, g

(<MomentumArray4D [[], [[{pt: 18.6, ... charge: -1}]]] type='299683 * var * var *...'>,
 <MomentumArray3D [[], [[{pt: 18.7, ... pdgId: 13}]]] type='299683 * var * var * ...'>)

In [40]:
mu.deltaR(g)

<Array [[], [[0.000383, ... 3.21, 1.25]]] type='299683 * var * var * float32'>

In [41]:
hist.Hist.new.Reg(100, 0, 5).Double().fill(
    ak.flatten(mu.deltaR(g), axis=None)
)

Hist(Regular(100, 0, 5, label='Axis 0'), storage=Double()) # Sum: 2962875.0 (2981136.0 with flow)

In [45]:
hist.Hist.new.Reg(100, 0, 5).Double().fill(
    ak.flatten(mu.deltaR(g)[abs(g.pdgId) == 13], axis=None)
)

Hist(Regular(100, 0, 5, label='Axis 0'), storage=Double()) # Sum: 2021989.0 (2023850.0 with flow)

In [46]:
hist.Hist.new.Reg(100, 0, 5).Double().fill(
    ak.flatten(ak.min(mu.deltaR(g), axis=-1), axis=None)
)

Hist(Regular(100, 0, 5, label='Axis 0'), storage=Double()) # Sum: 423736.0 (423743.0 with flow)

In [47]:
best = ak.argmin(mu.deltaR(g), axis=-1, keepdims=True)
best

<Array [[], [[0], [1]], ... [1], [2], [3]]] type='299683 * var * var * ?int64'>

In [48]:
reco_gen[best]

<Array [[], [[{muon: {, ... pdgId: 13}}]]] type='299683 * var * var * ?{"muon": ...'>

In [49]:
ak.flatten(reco_gen[best], axis=-1)[:4].tolist()

[[],
 [{'muon': {'pt': 18.583789825439453,
    'eta': -0.17873963713645935,
    'phi': 2.1292223930358887,
    'mass': 0.10565836727619171,
    'charge': 1},
   'gen': {'pt': 18.733409881591797,
    'eta': -0.17861033976078033,
    'phi': 2.1295831203460693,
    'pdgId': -13}},
  {'muon': {'pt': 23.630338668823242,
    'eta': 0.22412824630737305,
    'phi': -2.0946476459503174,
    'mass': 0.10565836727619171,
    'charge': -1},
   'gen': {'pt': 23.816869735717773,
    'eta': 0.22458051145076752,
    'phi': -2.0947155952453613,
    'pdgId': 13}}],
 [],
 [{'muon': {'pt': 26.678863525390625,
    'eta': -1.2300245761871338,
    'phi': -1.3949246406555176,
    'mass': 0.10565836727619171,
    'charge': -1},
   'gen': {'pt': 26.755929946899414,
    'eta': -1.2301405668258667,
    'phi': -1.3949953317642212,
    'pdgId': 13}},
  {'muon': {'pt': 21.356121063232422,
    'eta': 1.2668139934539795,
    'phi': 1.0259664058685303,
    'mass': 0.10565836727619171,
    'charge': 1},
   'gen': {'pt':

In [50]:
import numba as nb

In [51]:
starttime = time.time()

sumpt = np.zeros(len(array), np.float64)
for i, event in enumerate(array):
    for muon in event.muons:
        sumpt[i] += muon.pt

python_time = time.time() - starttime
print(f"total time: {python_time} sec")

total time: 37.60730266571045 sec


In [52]:
@nb.jit
def calculate_sumpt(array):
    out = np.zeros(len(array), np.float64)
    for i, event in enumerate(array):
        for muon in event.muons:
            out[i] += muon.pt
    return out

In [53]:
calculate_sumpt(array)

array([  0.        ,  42.21412849,   0.        , ...,  26.54553461,
        91.49959946, 108.89332008])

In [54]:
starttime = time.time()

sumpt = calculate_sumpt(array)

numba_time = time.time() - starttime
print(f"total time: {numba_time} sec")

total time: 0.0028619766235351562 sec


In [55]:
python_time / numba_time

13140.324891702765

In [58]:
starttime = time.time()

sumpt = ak.sum(array, axis=-1)

awkward_time = time.time() - starttime
print(f"total time: {awkward_time} sec")

total time: 0.08374428749084473 sec


In [59]:
python_time / awkward_time

449.07305074178146

In [60]:
@nb.jit
def build_nested(array, builder):
    for event in array:
        builder.begin_list()
        
        for muon in event.muons:
            builder.append(muon.pt)
        
        builder.end_list()
    
    return builder

build_nested(array, ak.ArrayBuilder()).snapshot()

<Array [[], [18.6, 23.6, ... 36.4, 19.6, 22.6]] type='299683 * var * float64'>

In [61]:
array.muons.pt

<Array [[], [18.6, 23.6, ... 36.4, 19.6, 22.6]] type='299683 * var * float32'>

In [74]:
@nb.jit
def matching(array_muons, array_gen, builder):
    for muons_event, gen_event in zip(array_muons, array_gen):
        builder.begin_list()

        for muon in muons_event:
            best_i = -1
            best_dr = -1.0
            for i, gen in enumerate(gen_event):
                dr = muon.deltaR(gen)
                if best_i < 0 or dr < best_dr:
                    best_i = i
                    best_dr = dr

            if best_i < 0:
                builder.append(None)
            else:
                builder.append(best_i)

        builder.end_list()

    return builder

index_of_best = matching(muons, gen, ak.ArrayBuilder()).snapshot()
index_of_best

<Array [[], [0, 1], [], ... 2], [0, 1, 2, 3]] type='299683 * var * int64'>

In [78]:
gen_match = gen[index_of_best]
gen_match

<MomentumArray3D [[], [{pt: 18.7, ... pdgId: 13}]] type='299683 * var * Momentum...'>

In [79]:
ak.num(gen), ak.num(muons), ak.num(gen_match)

(<Array [6, 6, 1, 5, 10, 2, ... 3, 7, 7, 6, 8] type='299683 * int64'>,
 <Array [0, 2, 0, 3, 1, 1, ... 5, 2, 0, 2, 3, 4] type='299683 * int64'>,
 <Array [0, 2, 0, 3, 1, 1, ... 5, 2, 0, 2, 3, 4] type='299683 * int64'>)

In [81]:
ak.zip({"muons": muons, "gen": gen_match})

<Array [[], [{muons: {, ... pdgId: 13}}]] type='299683 * var * {"muons": Momentu...'>